In [ ]:
# Instalasi Library
!pip install transformers torch accelerate bitsandbytes pandas -q
# print("don")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.1 MB/s eta 0:00:00


In [ ]:
# Import libs
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import pandas as pd
import time
import re

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Pemuatan Model
model_name = "Qwen/Qwen2-7B-Instruct"
print(f"download model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
print(f"don p2")

download model: Qwen/Qwen2-7B-Instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Device set to use cuda:0


don p2


# **PROMPT 1**

In [ ]:
# Definisi Variabel Global dan Fungsi
KATEGORI_ASPEK = "[Kamera, Baterai, Performa & Gaming, Layar, Desain & Kualitas Bodi, Harga, Pengiriman, Layanan Penjual, UI]"

# prompt
# def buat_prompt_zero_shot(ulasan):
#     return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat.
# Tugas Anda adalah mengidentifikasi semua aspek relevan dari ulasan pengguna berikut.
# Pilih satu atau lebih aspek dari daftar ini: {KATEGORI_ASPEK}
# anda hanya boleh memilih aspek dari daftar {KATEGORI_ASPEK} saja, tidak boleh ada aspek diluar kateegori tersebut.
# Format jawaban Anda HANYA berupa list Python. Jangan berikan penjelasan tambahan.
# Ulasan: "{ulasan}"
# Aspek:"""

def buat_prompt_zero_shot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat.
Tugas Anda adalah mengidentifikasi SEMUA aspek relevan dari ulasan pengguna berikut.
DAFTAR ASPEK RESMI: {KATEGORI_ASPEK}

PENTING:
- JANGAN pernah menyebut aspek di luar daftar di atas.
- Contoh aspek ILEGAL: "Kinerja", "Kualitas Barang", "Garansi", "Kondisi Barang" — ini BUKAN nama aspek resmi.
- Format jawaban HARUS berupa list Python TANPA penjelasan tambahan.
- Jawaban HARUS dimulai dengan `[` dan diakhiri `]`.

ULASAN: "{ulasan}"
ASPEK:"""

# def buat_prompt_few_shot(ulasan):
#     return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat.
# Tugas Anda adalah mengidentifikasi semua aspek relevan dari ulasan pengguna berdasarkan contoh.
# Daftar kategori aspek yang tersedia adalah: {KATEGORI_ASPEK}

# CONTOH:
# Ulasan: "Kamera depan jelek, baterai tahan seharian."
# Aspek: ["Kamera", "Baterai"]

# Ulasan: "Desainnya premium banget, harga terlalu mahal."
# Aspek: ["Desain", "Harga"]

# anda hanya boleh memilih aspek dari daftar {KATEGORI_ASPEK} saja, tidak boleh ada aspek diluar kateegori tersebut.
# Sekarang, kerjakan ulasan berikut.
# Ulasan: "{ulasan}"
# Format jawaban Anda HANYA berupa list Python.
# Aspek:"""

def buat_prompt_few_shot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat.
Tugas Anda adalah mengidentifikasi SEMUA aspek relevan dari ulasan pengguna berikut.
DAFTAR ASPEK RESMI: {KATEGORI_ASPEK}

PENTING:
- JANGAN pernah menyebut aspek di luar daftar di atas.
- Contoh aspek ILEGAL: "Kinerja", "Kualitas Barang", "Garansi" — ini BUKAN nama aspek resmi.
- Format jawaban HARUS berupa list Python TANPA penjelasan tambahan.
- Jawaban HARUS dimulai dengan `[` dan diakhiri `]`.

CONTOH:
Ulasan: "Kondisi Barang Baik, harga terlalu mahal, Penjual ramah."
Aspek: ["Desain & Kualitas Bodi", "Harga", Layanan Penjual]

Ulasan: "Kamera depan jelek, Kinerja mantap, baterai tahan seharian."
Aspek: ["Kamera", "Performa & Gaming", "Baterai"]

Ulasan: "Pengiriman cepat, Fungsi sesuai, Garansi ada."
Aspek: ["Pengiriman", "Performa & Gaming", "Layanan Penjual"]

Sekarang, kerjakan ulasan berikut.
Ulasan: "{ulasan}"
Aspek:"""

def buat_prompt_zero_shot_cot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat. Anda HARUS mengikuti instruksi ini dengan TEPAT.

TUGAS: Identifikasi SEMUA aspek relevan dari ulasan pengguna berikut. Anda HANYA BOLEH memilih aspek dari daftar ini: {KATEGORI_ASPEK}. JANGAN membuat aspek baru. JANGAN menambahkan keterangan tambahan.

LANGKAH-LANGKAH:
1. Baca ulasan dengan cermat.
2. Jika ulasan kosong atau tidak mengandung informasi tentang produk, langsung tulis: JAWABAN AKHIR: []
3. Jika ulasan mengandung informasi, identifikasi setiap aspek yang disebutkan — HANYA dari daftar di atas.
4. Abaikan semua kata di luar daftar aspek — misal: "Kondisi Barang", "Garansi", "Kinerja" — karena itu BUKAN nama aspek resmi.
5. Setelah selesai menganalisis, tulis jawaban AKHIR dalam format: JAWABAN AKHIR: [aspek1, aspek2, ...] — dan HENTIKAN segera setelah tanda "]" — JANGAN tambahkan apa pun setelahnya.

ULASAN: "{ulasan}"

PENALARAN LANGKAH DEMI LANGKAH:

JAWABAN AKHIR: ["""

def buat_prompt_few_shot_cot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat. Tugas Anda adalah mengidentifikasi semua aspek relevan dari ulasan pengguna berikut. Anda HANYA BOLEH memilih aspek dari daftar ini: {KATEGORI_ASPEK}. Jika frasa tidak secara eksplisit menyebut nama aspek, pilih aspek yang paling relevan dari daftar.

CONTOH:

Ulasan: "Kondisi Barang: Baru Original
Garansi: Original
Kinerja: Berfungsi dengan baik
Mantab barang yang ditunggu2 datang, packaging aman, barang baru dan bersebelahan, berfungsi dengan normal"
Identifikasi Frasa Kunci:
- "Baru Original", "barang baru" → Desain & Kualitas Bodi
- "Garansi: Original" → Layanan Penjual
- "Berfungsi dengan baik", "berfungsi dengan normal" → Performa & Gaming
- "packaging aman" → Pengiriman
Jawaban Akhir: ["Desain & Kualitas Bodi", "Layanan Penjual", "Performa & Gaming", "Pengiriman"]

Ulasan: "Barang sampai dengan selamat, packing rapih. HP-nya ngebut banget buat main ML, tapi agak cepet panas. Layar jernih, kamera depannya biasa aja."
Identifikasi Frasa Kunci:
- "Barang sampai dengan selamat, packing rapih" → Pengiriman
- "HP-nya ngebut banget buat main ML" → Performa & Gaming
- "Layar jernih" → Layar
- "kamera depannya biasa aja" → Kamera
Jawaban Akhir: ["Pengiriman", "Performa & Gaming", "Layar", "Kamera"]

Ulasan: "Kualitas: Best value phone in the market right now
Garansi: panjang
Kondisi Barang: 100% aman mulus
hands down sama Poco F7! Performanya mantep abis, multitasking lancar jaya, game berat juga nggak ngelag Layar AMOLED nya keren banget, warna-warna cerah dan tajam, bikin gue makin asik nonton YouTube atau main game, Kamera 50MP nya juga jos, Foto-foto jernih, detail, dan nggak kalah sama kamera profesional! Baterai 650MAH nya awet banget, gue bisa main game seharian tanpa perlu khawatir kehabisan daya. Dan pengisian cepat 90Watt ? Gue bisa balik main dalam waktu singkat, Harganya juga nggak bikin kantong jebol, worth it banget"
Identifikasi Frasa Kunci:
- "Performanya mantep abis, multitasking lancar jaya, game berat juga nggak ngelag" → Performa & Gaming
- "Layar AMOLED nya keren banget, warna-warna cerah dan tajam" → Layar
- "Kamera 50MP nya juga jos, Foto-foto jernih" → Kamera
- "Baterai 650MAH nya awet banget" → Baterai
- "Harganya juga nggak bikin kantong jebol" → Harga
Jawaban Akhir: ["Performa & Gaming", "Layar", "Kamera", "Baterai", "Harga"]

Sekarang, lakukan hal yang sama untuk ulasan berikut.
Ulasan: "{ulasan}"
Identifikasi Frasa Kunci:"""


def bersihkan_respons(teks_respons):
    """Membersihkan output mentah dari LLM — versi diperkuat."""
    # Daftar aspek resmi
    daftar_aspek = [
        "Kamera", "Baterai", "Performa & Gaming", "Layar",
        "Desain & Kualitas Bodi", "Harga", "Pengiriman", "Layanan Penjual", "UI"
    ]

    # Coba ekstrak isi dalam []
    pola = r'\[(.*?)\]'
    hasil_pencarian = re.search(pola, teks_respons, re.DOTALL)
    if hasil_pencarian:
        isi = hasil_pencarian.group(1)
        # Pisahkan aspek
        aspek_list = [a.strip().strip('"').strip("'") for a in isi.split(",") if a.strip()]
        # Filter hanya aspek yang valid
        aspek_valid = [a for a in aspek_list if a in daftar_aspek]
        return str(aspek_valid)
    else:
        # Jika tidak ada [], kembalikan list kosong
        return "[]"

def panggil_model_lokal(prompt):
    """Kirim satu prompt ke model dan mengembalikan hasilnya."""
    messages = [
        {"role": "system", "content": "Anda adalah seorang ahli analisis ulasan produk yang akurat."},
        {"role": "user", "content": prompt}
    ]
    prompt_formatted = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    outputs = pipe(
        prompt_formatted,
        max_new_tokens=16384,
        do_sample=False
    )
    # outputs = pipe(
    #     prompt_formatted,
    #     max_new_tokens=16384,
    #     do_sample=True,
    #     temperature=1,
    #     top_p=0.9
    # )


    hasil_teks = outputs[0]["generated_text"][len(prompt_formatted):]
    return hasil_teks

def ekstrak_jawaban_akhir_dari_cot(teks_respons_cot):
    """Mencari Jawaban Akhir, lalu bersihkan jadi list Python."""
    pola = r"Jawaban\s*Akhir(?: Contoh)?:?\s*(\[.*?\])"
    hasil_pencarian = re.search(pola, teks_respons_cot, re.IGNORECASE | re.DOTALL)

    if hasil_pencarian:
        return bersihkan_respons(hasil_pencarian.group(1))
    else:
        return bersihkan_respons(teks_respons_cot)


print("Semua fungsi 'perkakas' berhasil didefinisikan.")

Semua fungsi 'perkakas' berhasil didefinisikan.


# **PROMPT 2**

In [ ]:
# Definisi Variabel Global dan Fungsi
KATEGORI_ASPEK = "[Kamera, Baterai, Performa & Gaming, Layar, Desain & Kualitas Bodi, Harga, Pengiriman, Layanan Penjual, UI]"

# Prompt Zero-shot biasa (tidak berubah)
def buat_prompt_zero_shot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat.
Tugas Anda adalah mengidentifikasi SEMUA aspek relevan dari ulasan pengguna berikut.
DAFTAR ASPEK RESMI: {KATEGORI_ASPEK}

PENTING:
- JANGAN pernah menyebut aspek di luar daftar di atas.
- Contoh aspek ILEGAL: "Kinerja", "Kualitas Barang", "Garansi", "Kondisi Barang" — ini BUKAN nama aspek resmi.
- Format jawaban HARUS berupa list Python TANPA penjelasan tambahan.
- Jawaban HARUS dimulai dengan `[` dan diakhiri `]`.

ULASAN: "{ulasan}"
ASPEK:"""

# Prompt Few-shot biasa (tidak berubah)
def buat_prompt_few_shot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat.
Tugas Anda adalah mengidentifikasi SEMUA aspek relevan dari ulasan pengguna berikut.
DAFTAR ASPEK RESMI: {KATEGORI_ASPEK}

PENTING:
- JANGAN pernah menyebut aspek di luar daftar di atas.
- Contoh aspek ILEGAL: "Kinerja", "Kualitas Barang", "Garansi" — ini BUKAN nama aspek resmi.
- Format jawaban HARUS berupa list Python TANPA penjelasan tambahan.
- Jawaban HARUS dimulai dengan `[` dan diakhiri `]`.

CONTOH:
Ulasan: "Kamera depan jelek, baterai tahan seharian."
Aspek: ["Kamera", "Baterai"]

Ulasan: "Desainnya premium banget, harga terlalu mahal."
Aspek: ["Desain & Kualitas Bodi", "Harga"]

Ulasan: "Pengiriman cepat, layanan penjual ramah."
Aspek: ["Pengiriman", "Layanan Penjual"]

Sekarang, kerjakan ulasan berikut.
Ulasan: "{ulasan}"
Aspek:"""

# Prompt Zero-shot CoT — TAHAP 1: TRIGGER PENALARAN
def buat_prompt_zero_shot_cot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat. Tugas Anda adalah mengidentifikasi semua aspek relevan dari ulasan pengguna berikut. Anda HANYA BOLEH memilih aspek dari daftar ini: {KATEGORI_ASPEK}. Jika frasa tidak secara eksplisit menyebut nama aspek, pilih aspek yang paling relevan dari daftar.

ULASAN: "{ulasan}"

Mari berpikir langkah demi langkah."""

# Prompt Ekstraksi — TAHAP 2: EKSTRAK JAWABAN DARI PENALARAN
def buat_prompt_ekstrak_dari_penalaran(penalaran):
    return f"""Berdasarkan penalaran berikut, identifikasi aspek yang disebutkan. Hanya pilih dari daftar: {KATEGORI_ASPEK}. Jawaban harus dalam format list Python.

Penalaran:
{penalaran}

Jawaban akhir:"""

# Prompt Few-shot CoT (tidak berubah)
def buat_prompt_few_shot_cot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat. Tugas Anda adalah mengidentifikasi semua aspek relevan dari ulasan pengguna berikut. Anda HANYA BOLEH memilih aspek dari daftar ini: {KATEGORI_ASPEK}. Jika frasa tidak secara eksplisit menyebut nama aspek, pilih aspek yang paling relevan dari daftar.

CONTOH:

Ulasan: "Kualitas: Best value phone in the market right now
Garansi: panjang
Kondisi Barang: 100% aman mulus
hands down sama Poco F7! Performanya mantep abis, multitasking lancar jaya, game berat juga nggak ngelag Layar AMOLED nya keren banget, warna-warna cerah dan tajam, bikin gue makin asik nonton YouTube atau main game, Kamera 50MP nya juga jos, Foto-foto jernih, detail, dan nggak kalah sama kamera profesional! Baterai 650MAH nya awet banget, gue bisa main game seharian tanpa perlu khawatir kehabisan daya. Dan pengisian cepat 90Watt ? Gue bisa balik main dalam waktu singkat, Harganya juga nggak bikin kantong jebol, worth it banget"
Identifikasi Frasa Kunci:
- "Performanya mantep abis, multitasking lancar jaya, game berat juga nggak ngelag" → Performa & Gaming
- "Layar AMOLED nya keren banget, warna-warna cerah dan tajam" → Layar
- "Kamera 50MP nya juga jos, Foto-foto jernih" → Kamera
- "Baterai 650MAH nya awet banget" → Baterai
- "Harganya juga nggak bikin kantong jebol" → Harga
Jawaban Akhir: ["Performa & Gaming", "Layar", "Kamera", "Baterai", "Harga"]

Ulasan: "Kondisi Barang: Baru Original
Garansi: Original
Kinerja: Berfungsi dengan baik
Mantab barang yang ditunggu2 datang, packaging aman, barang baru dan bersebelahan, berfungsi dengan normal"
Identifikasi Frasa Kunci:
- "Baru Original", "barang baru" → Desain & Kualitas Bodi
- "Garansi: Original" → Layanan Penjual
- "Berfungsi dengan baik", "berfungsi dengan normal" → Performa & Gaming
- "packaging aman" → Pengiriman
Jawaban Akhir: ["Desain & Kualitas Bodi", "Layanan Penjual", "Performa & Gaming", "Pengiriman"]

Ulasan: "Barang sampai dengan selamat, packing rapih. HP-nya ngebut banget buat main ML, tapi agak cepet panas. Layar jernih, kamera depannya biasa aja."
Identifikasi Frasa Kunci:
- "Barang sampai dengan selamat, packing rapih" → Pengiriman
- "HP-nya ngebut banget buat main ML" → Performa & Gaming
- "Layar jernih" → Layar
- "kamera depannya biasa aja" → Kamera
Jawaban Akhir: ["Pengiriman", "Performa & Gaming", "Layar", "Kamera"]

Sekarang, lakukan hal yang sama untuk ulasan berikut.
Ulasan: "{ulasan}"
Identifikasi Frasa Kunci:"""

In [ ]:
# Fungsi pembersih respons (tidak berubah)
def bersihkan_respons(teks_respons):
    """Membersihkan output mentah dari LLM — versi diperkuat."""
    daftar_aspek = [
        "Kamera", "Baterai", "Performa & Gaming", "Layar",
        "Desain & Kualitas Bodi", "Harga", "Pengiriman", "Layanan Penjual", "UI"
    ]
    pola = r'\[(.*?)\]'
    hasil_pencarian = re.search(pola, teks_respons, re.DOTALL)
    if hasil_pencarian:
        isi = hasil_pencarian.group(1)
        aspek_list = [a.strip().strip('"').strip("'") for a in isi.split(",") if a.strip()]
        aspek_valid = [a for a in aspek_list if a in daftar_aspek]
        return str(aspek_valid)
    else:
        return "[]"

# Fungsi panggil model dengan sampling (untuk penalaran)
def panggil_model_sampling(prompt):
    messages = [
        {"role": "system", "content": "Anda adalah seorang ahli analisis ulasan produk yang akurat."},
        {"role": "user", "content": prompt}
    ]
    prompt_formatted = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(
        prompt_formatted,
        max_new_tokens=16384,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    hasil_teks = outputs[0]["generated_text"][len(prompt_formatted):]
    return hasil_teks

# Fungsi panggil model dengan greedy (untuk ekstraksi)
def panggil_model_greedy(prompt):
    messages = [
        {"role": "system", "content": "Anda adalah seorang ahli analisis ulasan produk yang akurat."},
        {"role": "user", "content": prompt}
    ]
    prompt_formatted = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(
        prompt_formatted,
        max_new_tokens=16384,
        do_sample=False
    )
    hasil_teks = outputs[0]["generated_text"][len(prompt_formatted):]
    return hasil_teks

def zero_shot_cot_dengan_ekstraksi(ulasan):
    # Tahap 1: Dapatkan penalaran
    prompt_penalaran = buat_prompt_zero_shot_cot(ulasan)
    penalaran = panggil_model_sampling(prompt_penalaran)

    # Jika penalaran kosong, kembalikan []
    if not penalaran.strip():
        return "[]"

    # Tahap 2: Ekstrak jawaban dari penalaran
    prompt_ekstrak = buat_prompt_ekstrak_dari_penalaran(penalaran)
    jawaban_mentah = panggil_model_greedy(prompt_ekstrak)
    return bersihkan_respons(jawaban_mentah)

# Fungsi panggil model lokal (untuk Zero-shot & Few-shot biasa)
def panggil_model_lokal(prompt):
    messages = [
        {"role": "system", "content": "Anda adalah seorang ahli analisis ulasan produk yang akurat."},
        {"role": "user", "content": prompt}
    ]
    prompt_formatted = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(
        prompt_formatted,
        max_new_tokens=16384,
        do_sample=False
    )
    hasil_teks = outputs[0]["generated_text"][len(prompt_formatted):]
    return hasil_teks

# Fungsi ekstrak jawaban akhir dari CoT (untuk Few-shot CoT)
def ekstrak_jawaban_akhir_dari_cot(teks_respons_cot):
    pola = r"Jawaban\s*Akhir(?: Contoh)?:?\s*(\[.*?\])"
    hasil_pencarian = re.search(pola, teks_respons_cot, re.IGNORECASE | re.DOTALL)
    if hasil_pencarian:
        return bersihkan_respons(hasil_pencarian.group(1))
    else:
        return bersihkan_respons(teks_respons_cot)

print("Semua fungsi 'perkakas' berhasil didefinisikan.")

Semua fungsi 'perkakas' berhasil didefinisikan.


# **Proses Eksperimen (Satu per Satu)**

In [ ]:
# Proses Eksperimen
nama_file_input = 'development_set_berfitur.csv'
nama_file_output = f'eksperimen_terbaru_1.csv'

df_dev = pd.read_csv(nama_file_input)
df_dev_sample = df_dev.head(850).copy()
print(f"Memulai eksperimen pada {len(df_dev_sample)} sampel data...")

# Inisialisasi kolom prediksi + waktu
df_dev_sample['prediksi_zero_shot'] = ""
df_dev_sample['waktu_zs_detik'] = 0.0

df_dev_sample['prediksi_few_shot'] = ""
df_dev_sample['waktu_fs_detik'] = 0.0

df_dev_sample['prediksi_zero_shot_cot'] = ""
df_dev_sample['waktu_zsc_detik'] = 0.0

df_dev_sample['prediksi_few_shot_cot'] = ""
df_dev_sample['waktu_fsc_detik'] = 0.0

Memulai eksperimen pada 842 sampel data...


In [ ]:
# # ZERO-SHOT
# print("\n Memproses prompt Zero-shot")
# for index, row in df_dev_sample.iterrows():
#     print(f"  -> Mengirim Ulasan #{index+1}...")
#     prompt = buat_prompt_zero_shot(row['comment'])
#     hasil = panggil_model_lokal(prompt)
#     df_dev_sample.loc[index, 'prediksi_zero_shot'] = bersihkan_respons(hasil)

# ZERO-SHOT
print("\n Memproses prompt Zero-shot")
for index, row in df_dev_sample.iterrows():
    print(f"  -> Mengirim Ulasan #{index+1}...")

    start_time = time.time()
    prompt = buat_prompt_zero_shot(row['comment'])
    hasil = panggil_model_lokal(prompt)
    durasi = round(time.time() - start_time, 2)

    df_dev_sample.loc[index, 'prediksi_zero_shot'] = bersihkan_respons(hasil)
    df_dev_sample.loc[index, 'waktu_zs_detik'] = durasi


 Memproses prompt Zero-shot
  -> Mengirim Ulasan #1...
  -> Mengirim Ulasan #2...
  -> Mengirim Ulasan #3...
  -> Mengirim Ulasan #4...
  -> Mengirim Ulasan #5...
  -> Mengirim Ulasan #6...
  -> Mengirim Ulasan #7...
  -> Mengirim Ulasan #8...
  -> Mengirim Ulasan #9...
  -> Mengirim Ulasan #10...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


  -> Mengirim Ulasan #11...
  -> Mengirim Ulasan #12...
  -> Mengirim Ulasan #13...
  -> Mengirim Ulasan #14...
  -> Mengirim Ulasan #15...
  -> Mengirim Ulasan #16...
  -> Mengirim Ulasan #17...
  -> Mengirim Ulasan #18...
  -> Mengirim Ulasan #19...
  -> Mengirim Ulasan #20...
  -> Mengirim Ulasan #21...
  -> Mengirim Ulasan #22...
  -> Mengirim Ulasan #23...
  -> Mengirim Ulasan #24...
  -> Mengirim Ulasan #25...
  -> Mengirim Ulasan #26...
  -> Mengirim Ulasan #27...
  -> Mengirim Ulasan #28...
  -> Mengirim Ulasan #29...
  -> Mengirim Ulasan #30...
  -> Mengirim Ulasan #31...
  -> Mengirim Ulasan #32...
  -> Mengirim Ulasan #33...
  -> Mengirim Ulasan #34...
  -> Mengirim Ulasan #35...
  -> Mengirim Ulasan #36...
  -> Mengirim Ulasan #37...
  -> Mengirim Ulasan #38...
  -> Mengirim Ulasan #39...
  -> Mengirim Ulasan #40...
  -> Mengirim Ulasan #41...
  -> Mengirim Ulasan #42...
  -> Mengirim Ulasan #43...
  -> Mengirim Ulasan #44...
  -> Mengirim Ulasan #45...
  -> Mengirim Ulasan

In [ ]:
# # FEW-SHOT
# print("\n--- Memproses prompt Few-shot ---")
# for index, row in df_dev_sample.iterrows():
#     print(f"  -> Mengirim Ulasan #{index+1}...")
#     prompt = buat_prompt_few_shot(row['comment'])
#     hasil = panggil_model_lokal(prompt)
#     df_dev_sample.loc[index, 'prediksi_few_shot'] = bersihkan_respons(hasil)

# FEW-SHOT
print("\n--- Memproses prompt Few-shot ---")
for index, row in df_dev_sample.iterrows():
    print(f"  -> Mengirim Ulasan #{index+1}...")

    start_time = time.time()
    prompt = buat_prompt_few_shot(row['comment'])
    hasil = panggil_model_lokal(prompt)
    durasi = round(time.time() - start_time, 2)

    df_dev_sample.loc[index, 'prediksi_few_shot'] = bersihkan_respons(hasil)
    df_dev_sample.loc[index, 'waktu_fs_detik'] = durasi

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Memproses prompt Few-shot ---
  -> Mengirim Ulasan #1...
  -> Mengirim Ulasan #2...
  -> Mengirim Ulasan #3...
  -> Mengirim Ulasan #4...
  -> Mengirim Ulasan #5...
  -> Mengirim Ulasan #6...
  -> Mengirim Ulasan #7...
  -> Mengirim Ulasan #8...
  -> Mengirim Ulasan #9...
  -> Mengirim Ulasan #10...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


  -> Mengirim Ulasan #11...
  -> Mengirim Ulasan #12...
  -> Mengirim Ulasan #13...
  -> Mengirim Ulasan #14...
  -> Mengirim Ulasan #15...
  -> Mengirim Ulasan #16...
  -> Mengirim Ulasan #17...
  -> Mengirim Ulasan #18...
  -> Mengirim Ulasan #19...
  -> Mengirim Ulasan #20...
  -> Mengirim Ulasan #21...
  -> Mengirim Ulasan #22...
  -> Mengirim Ulasan #23...
  -> Mengirim Ulasan #24...
  -> Mengirim Ulasan #25...
  -> Mengirim Ulasan #26...
  -> Mengirim Ulasan #27...
  -> Mengirim Ulasan #28...
  -> Mengirim Ulasan #29...
  -> Mengirim Ulasan #30...
  -> Mengirim Ulasan #31...
  -> Mengirim Ulasan #32...
  -> Mengirim Ulasan #33...
  -> Mengirim Ulasan #34...
  -> Mengirim Ulasan #35...
  -> Mengirim Ulasan #36...
  -> Mengirim Ulasan #37...
  -> Mengirim Ulasan #38...
  -> Mengirim Ulasan #39...
  -> Mengirim Ulasan #40...
  -> Mengirim Ulasan #41...
  -> Mengirim Ulasan #42...
  -> Mengirim Ulasan #43...
  -> Mengirim Ulasan #44...
  -> Mengirim Ulasan #45...
  -> Mengirim Ulasan

In [ ]:
# ZERO-SHOT CoT
print("\n--- Memproses prompt Zero-shot CoT ---")
for index, row in df_dev_sample.iterrows():
    print(f"  -> Mengirim Ulasan #{index+1}...")
    hasil = zero_shot_cot_dengan_ekstraksi(row['comment'])
    df_dev_sample.loc[index, 'prediksi_zero_shot_cot'] = hasil


--- Memproses prompt Zero-shot CoT ---
  -> Mengirim Ulasan #1...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Mengirim Ulasan #2...
  -> Mengirim Ulasan #3...
  -> Mengirim Ulasan #4...
  -> Mengirim Ulasan #5...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


  -> Mengirim Ulasan #6...
  -> Mengirim Ulasan #7...
  -> Mengirim Ulasan #8...
  -> Mengirim Ulasan #9...
  -> Mengirim Ulasan #10...
  -> Mengirim Ulasan #11...
  -> Mengirim Ulasan #12...
  -> Mengirim Ulasan #13...
  -> Mengirim Ulasan #14...
  -> Mengirim Ulasan #15...
  -> Mengirim Ulasan #16...
  -> Mengirim Ulasan #17...
  -> Mengirim Ulasan #18...
  -> Mengirim Ulasan #19...
  -> Mengirim Ulasan #20...
  -> Mengirim Ulasan #21...
  -> Mengirim Ulasan #22...
  -> Mengirim Ulasan #23...
  -> Mengirim Ulasan #24...
  -> Mengirim Ulasan #25...
  -> Mengirim Ulasan #26...
  -> Mengirim Ulasan #27...
  -> Mengirim Ulasan #28...
  -> Mengirim Ulasan #29...
  -> Mengirim Ulasan #30...
  -> Mengirim Ulasan #31...
  -> Mengirim Ulasan #32...
  -> Mengirim Ulasan #33...
  -> Mengirim Ulasan #34...
  -> Mengirim Ulasan #35...
  -> Mengirim Ulasan #36...
  -> Mengirim Ulasan #37...
  -> Mengirim Ulasan #38...
  -> Mengirim Ulasan #39...
  -> Mengirim Ulasan #40...
  -> Mengirim Ulasan #41

In [ ]:
# FEW-SHOT CoT
print("\n--- Memproses prompt Few-shot CoT ---")
for index, row in df_dev_sample.iterrows():
    print(f"  -> Mengirim Ulasan #{index+1}...")
    prompt = buat_prompt_few_shot_cot(row['comment'])
    hasil_panjang = panggil_model_lokal(prompt)
    df_dev_sample.loc[index, 'prediksi_few_shot_cot'] = ekstrak_jawaban_akhir_dari_cot(hasil_panjang)


--- Memproses prompt Few-shot CoT ---
  -> Mengirim Ulasan #1...
  -> Mengirim Ulasan #2...
  -> Mengirim Ulasan #3...
  -> Mengirim Ulasan #4...
  -> Mengirim Ulasan #5...
  -> Mengirim Ulasan #6...
  -> Mengirim Ulasan #7...
  -> Mengirim Ulasan #8...
  -> Mengirim Ulasan #9...
  -> Mengirim Ulasan #10...
  -> Mengirim Ulasan #11...
  -> Mengirim Ulasan #12...
  -> Mengirim Ulasan #13...
  -> Mengirim Ulasan #14...
  -> Mengirim Ulasan #15...
  -> Mengirim Ulasan #16...
  -> Mengirim Ulasan #17...
  -> Mengirim Ulasan #18...
  -> Mengirim Ulasan #19...
  -> Mengirim Ulasan #20...
  -> Mengirim Ulasan #21...
  -> Mengirim Ulasan #22...
  -> Mengirim Ulasan #23...
  -> Mengirim Ulasan #24...
  -> Mengirim Ulasan #25...
  -> Mengirim Ulasan #26...
  -> Mengirim Ulasan #27...
  -> Mengirim Ulasan #28...
  -> Mengirim Ulasan #29...
  -> Mengirim Ulasan #30...
  -> Mengirim Ulasan #31...
  -> Mengirim Ulasan #32...
  -> Mengirim Ulasan #33...
  -> Mengirim Ulasan #34...
  -> Mengirim Ulas

In [ ]:
# SAVE
print("\nEksperimen Selesai!")
df_dev_sample.to_csv('eksperimen_FS_1.csv', index=False)
print(f"Hasil berhasil disimpan sebagai ''eksperimen_ZS_1.csv''")


Eksperimen Selesai!
Hasil berhasil disimpan sebagai ''eksperimen_ZS_1.csv''


In [ ]:
TEST

Prompt yang akan dibongkar:
Anda adalah seorang analis ulasan produk smartphone yang akurat.
Tugas Anda adalah mengidentifikasi semua aspek relevan dari ulasan pengguna berikut.
Pilih satu atau lebih aspek dari daftar ini: [Kamera, Baterai, Performa & Gaming, Layar, Desain, Harga, Pengiriman, Layanan Penjual, UI]
Ulasan: "Kameranya bagus, layar jernih."
Format jawaban Anda HANYA berupa list Python. Jangan berikan penjelasan tambahan.
Aspek:


--- HASIL TOKENISASI ---
Jumlah Token: 123

Input IDs (Setiap kata/simbol diubah menjadi angka ID unik dari kamus Qwen2):
tensor([[ 3036,    64, 37560,   511, 84168,  3181,   285,  8557, 45388, 60874,
         21511, 10371, 17324, 90198,   624,    51, 57831, 39138, 37560, 19729,
          1713, 77350, 77497,   438, 87176, 78315,   276, 24200,  8557, 45388,
         36686, 61275, 10207, 88994,   624,    47, 46968, 58566, 38372, 49935,
           438, 87176, 24200,  2994, 46201, 16806,    25,   508,    42,  3436,
            11,   425,   962,  2143,    11, 25001,    64,   609, 30462,    11,
         43924,   277,    11,  3874,   466,    11,   472, 17922,    11, 51050,
           404, 39210,    11, 43924, 28618, 13294, 65029,    11,  3689,   921,
            52, 14493,   276,    25,   330,    42, 15232, 24074,  8968,   355,
            11, 10962,   277,   502,   932,  6996, 10040,  4061, 16535, 38165,
         39138,   472, 17293,    32, 10207, 45831,  1140, 13027,    13,   619,
         


--- HASIL EMBEDDING ---
Dimensi Vektor Hasil Embedding:
torch.Size([1, 123, 3584])

Artinya: 1 batch, berisi 123 token, dimana setiap token sekarang direpresentasikan oleh sebuah vektor dengan 3584 angka.

Contoh Vektor Embedding untuk token PERTAMA (angka-angkanya adalah representasi makna dari token tsb):
tensor([-0.0044,  0.0120, -0.0148,  0.0099,  0.0035, -0.0183,  0.0018, -0.0059,
         0.0020, -0.0035], device='cuda:0', dtype=torch.float16)


In [ ]:
# function ClickConnect(){
#   console.log("Working to prevent timeout...");
#   document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
# }
# setInterval(ClickConnect, 60000);


# function ClickConnect(){
#   console.log("Klik");
#   document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
# }
# setInterval(ClickConnect, 50000);

# function ClickConnect(){
#   console.log("Geser");
#   document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
# }
# setInterval(ClickConnect, 40000);

# function ClickConnect(){
#   console.log("TEST CHECK");
#   document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
# }
# setInterval(ClickConnect, 70000);